# Traversability Simulation
## Stucture

```
.
├── agent
│   ├── Agent.py
│   ├── callbacks
│   │   ├── AgentCallback.py
│   │   └── RosBagSaver.py
│   ├── __init__.py
│   └── RospyAgent.py
├── data
├── docker
├── docker-compose.yml
├── env
│   ├── conditions
│   │   ├── __init__.py
│   │   └── __pycache__
│   ├── __init__.py
    ├── krock
│   │   ├── Krock.py
│   │   ├── KrockWebotsEnv.py
│   ├── __pycache__
│   │   └── __init__.cpython-36.pyc
│   └── webots
│       ├── utils.py
│       ├── WebotsEnv.py
│       └── WebotsWorld.py
├── example.ipynb
├── gym.ipynb
├── __init__.py
├── log
├── main.py
├── maps
├── notebooks
│   ├── test.bag
│   └── Visualisation.ipynb
├── parser.py
├── protocols
├── README.md
├── requirements.txt
├── start.sh
└── utils
    ├── History.py
    └── webots2ros
        ├── Supervisor.py

```

## Webots + Krock
![alt text](https://raw.githubusercontent.com/FrancescoSaverioZuppichini/Master-Thesis/gym/resources/images/krock.jpg")

In order to run the Krock robot using Webots we provide a [gym](https://gym.openai.com/) env. If you know gym, then you are good to go. It follows a **basic example** (API could change)

In [1]:
import rospy
import pprint

from env.webots.krock import Krock, KrockWebotsEnv

WORLD_PATH = '/home/francesco/Documents/Master-Thesis/core/env/webots/krock/krock2_ros/worlds/holes2.wbt'
EPISODES = 25

rospy.init_node("traversability_simulation")
# create our env
env = KrockWebotsEnv.from_file(WORLD_PATH)
# spawn the robot at a random location
init_obs = env.reset()

print('Initial observations:')
pprint.pprint(init_obs)

for _ in range(EPISODES):
#   go forward for a while
    obs, r, done, _ = env.step(env.GO_FORWARD)
    pprint.pprint(obs)
    if done: break
        
env.step(env.STOP)

/home/francesco/Documents/Master-Thesis/core/env/webots/krock/krock2_ros/worlds/holes2.wbt
Initial observations:
{'sensors': {'front_cam': None,
             'orientation': {'w': -2.5216734395810323,
                             'x': 7.173338268206636e-05,
                             'y': 0.9999999971706306,
                             'z': 2.2650841092634077e-05},
             'position': {'x': 2.6889421226713575,
                          'y': -3.6807569913877285,
                          'z': 0.4942669181354657}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': -2.5221118243535807,
                             'x': -6.216395821302278e-05,
                             'y': 0.9999998443585398,
                             'z': -0.0005544533690916926},
             'position': {'x': 2.688969389807841,
                          'y': -3.680806709380935,
                          'z': 0.41071718924640893}}}
{'sensors': {'front_cam': None,
             'orientation': {

{'sensors': {'front_cam': None,
             'orientation': {'w': -2.68214440160477,
                             'x': 0.012073799951991097,
                             'y': 0.9999082181547343,
                             'z': -0.006146431594297},
             'position': {'x': 2.3083462067411276,
                          'y': -3.3570657815394678,
                          'z': 0.12006390887250948}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': -2.671702136437405,
                             'x': -0.011375831098151538,
                             'y': 0.9997040910464141,
                             'z': 0.021501646725058745},
             'position': {'x': 2.2843681835849727,
                          'y': -3.3294284664613865,
                          'z': 0.10721098535533376}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': -2.6173130217910114,
                             'x': -0.005118571790742075,
                             'y': 0.99

({'sensors': {'position': {'x': 2.2028649318189197,
    'y': -3.2719038864493566,
    'z': 0.12060022390564128},
   'orientation': {'x': 0.03263725754272792,
    'y': 0.9994309209404891,
    'z': 0.008523126664270998,
    'w': -2.4154291941539587},
   'front_cam': None}},
 0,
 False,
 {})

`KrockWebotsEnv` implements `WebotsEnv` that is a generic env for Webots. 

Due to gym interface we can explore the action space and the observation space easily

In [2]:
print(env.action_space)
print(env.observation_space)

Dict(frontal_freq:Box(), lateral_freq:Box())
Dict(sensors:Dict(front_cam:Tuple(Box(600, 800, 3), Box(600, 800, 3)), orientation:Dict(w:Box(1,), x:Box(1,), y:Box(1,), z:Box(1,)), position:Dict(x:Box(1,), y:Box(1,), z:Box(1,))))


Perfect a random action is even esier

In [10]:
obs, r, done, _ = env.step(env.action_space.sample())
env.step(env.STOP)

({'sensors': {'position': {'x': 2.2800753279422543,
    'y': -3.031360041337963,
    'z': 0.10485295775513392},
   'orientation': {'x': -0.0032172121360495897,
    'y': 0.9994733477624561,
    'z': 0.03229050415493334,
    'w': -1.9883613196437562},
   'front_cam': None}},
 0,
 False,
 {})

If the camera throttle node is activated, `rosrun topic_tools throttle messages /krock/front_camera/image 30` calling `env.render()` will popup a window with the images from the krock's camera

## Agent

The `Agent` class describes something that interacts with the enviroment. It provides a nice interface to adopt.

### Usage

In [3]:
from agent import Agent

class DummyAgent(Agent):
    def act(self, env, *args, **kwargs):
        print('I am moving!')
    
a = DummyAgent()
a()

#### Lifecycle

The agent lifecyle is the following

In [4]:
a() # init
a.act(env) # do something
a.die(env) # RIP, clean up (e.g store stuff empty the state)

I am moving!


You need to first create the agent class and then call it to correctly initialise everything. You can think of it like a *lazy* loader.

### ROS
A `Rospy` agent is already implemented and can be found at `agent.RospyAgent`. It adds two methods to the basic `Agent`: `init_publishers` and `init_subscribers`. They must be overrided to initialise the subscribers and listeners


In [5]:
from agent import RospyAgent
from geometry_msgs.msg import PoseStamped
from sensor_msgs.msg import Joy

class MyRospyAgent(RospyAgent):
    def init_publishers(self):
        return {
            'joy': rospy.Publisher('/joy', Joy, queue_size=1),
        }

    def init_subscribers(self):
        return { 'pose': 
                rospy.Subscriber('/pose', PoseStamped, self.callback_pose)
               }

    def callback_pose(self, data):
        # this will triggers the `on_state_change` method in all callbacks
        self.state['pose'] = data

a = MyRospyAgent()
a()
print(a.subscribers)
print(a.publishers)

{'pose': <rospy.topics.Subscriber object at 0x7fea4095ab00>}
{'joy': <rospy.topics.Publisher object at 0x7fea54def400>}


### Callbacks
`Agent` implements the protocol `Callbackable` protocol. A callback can be
added by calling the `add_callback` or `add_callbacks`. 

The following interface expose the callback methods for `Agent`:

```python
class AgentCallback():
    def on_state_change(self, agent, key, value):
        pass

    def on_shut_down(self):
        pass
```

In [6]:
from agent.callbacks import AgentCallback

class MyCallback(AgentCallback):
    def on_state_change(self, agent, key, value):
        print(key, value)

Each `Agent` has a `state` that is just a `dict` . Every time it is changed it triggers the `on_state_change` event. This is usefull
to record stuff.

In [7]:
a = DummyAgent()
a.add_callback(MyCallback())
a()

a.state['foo'] = 'foo'

foo foo


#### RosBagSaver

A callback to store the topics of an agent into bags file is implemented

In [8]:
from agent.callbacks import RosBagSaver

a = MyRospyAgent()
a.add_callbacks([RosBagSaver(save_dir='/temp/data', topics=['pose'])])
a()

The files will be stored everytime `agent.die()` il called. A new thread is always spawned to avoid blocking